In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/scoot/?sortby=post_date%3ADesc&pagesize=100'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    for name, rating in zip(names[1:], ratings):
        review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'scoot'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']

# Save to CSV
df.to_csv('Data/scoot.csv',index=False)

In [3]:
# Sanity check
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,scoot,Paul Morel,2020-05-01,Not Verified | Returning as cheaply as possib...,Solo Leisure,Economy Class,10,4,2,NaN,NaN,5,5,yes
1,scoot,Constance Crampton,2020-04-26,Not Verified | We have used Scoot before and ...,Couple Leisure,Economy Class,2,NaN,NaN,NaN,NaN,NaN,1,no
2,scoot,Mick Tobor,2020-04-15,✅ Trip Verified | On our way to Bangkok airpo...,Couple Leisure,Economy Class,1,NaN,NaN,NaN,NaN,1,1,no
3,scoot,T Gan,2020-03-20,✅ Trip Verified | Melbourne to Hong Kong via ...,Solo Leisure,Economy Class,1,NaN,NaN,NaN,NaN,1,1,no
4,scoot,Francis Wong,2020-03-13,✅ Trip Verified | Due to severe reduction of ...,Solo Leisure,Economy Class,4,1,2,NaN,NaN,1,3,no


In [4]:
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
95,scoot,Amari Page,2019-03-25,✅ Trip Verified | Hong Kong to Melbourne via ...,Solo Leisure,Economy Class,1,1,1,1,1,1,1,no
96,scoot,S Deane,2019-03-14,✅ Trip Verified | Perth to Bangkok via Singap...,Solo Leisure,Business Class,10,4,5,3,NaN,5,5,yes
97,scoot,Darren Batt,2019-03-14,✅ Trip Verified | Gold Coast to Manila via Si...,Family Leisure,Economy Class,1,NaN,NaN,NaN,NaN,NaN,1,no
98,scoot,Timothy Irvin,2019-03-07,"✅ Trip Verified | Seated in 5C, in a supposedl...",Solo Leisure,Economy Class,3,2,3,2,NaN,4,3,no
99,scoot,P Kerai,2019-03-06,✅ Trip Verified | Perth to Phuket via Singapo...,Family Leisure,Economy Class,10,4,5,NaN,NaN,4,5,yes
